In [3]:
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
from plotly.graph_objs import *
from ggplot import *
from collections import Counter

project_id = "the-d-team-164512"
init_notebook_mode(connected=True)

## To Do
1. Select aggregation lvl (for diabetes probably per patient and year)
2. Manufacture features needed for the models (e.g. avg money spent for drugs or is_taking_drug for 5 most popular drugs among diabetic that are not popular in other groups). Normalize the features to have 0 mean and stdev 1.
3. Write validation function according to the metric used in leaderboard (AUC, see questions) to use in checking how well the model is doing.
4. Create classification of patients using diabetes drugs (1 - if used in a year, 0 if not). Final prediction is about patients in 2016, however we can also use previous years to have more models and ensemble them later.
5. Create logistic regression model to classify based on previous year input whether the patient will develop diabetes next year. The last layer should not omit the step of choosing as 1 the class with higher probability, instead it should return the probability of diabetes class itself.
6. Create another model that uses the whole history to predict 2016.
7. Use the same approach to create random forest model.
8. Create a few layers deep (4?/5?) fully connected NN with dropout layers and l2 regularization. Remember about correct initializing (std = 2/sqrt(n), n  - number of neurons
9. Create RNN(LSTM) with yearly inputs (?)
10. Make an ensemble of created models.
11. Additional models? - xgboost

 
I thought also about testing a bit more regarding store profitability and how it's influenced by main drugs sold. I thought about dividing the stores into few classes (negative, below 5k, 5k-40k, 40k-100k, 100k-200k, above) and checking the differences between them. Visualizations to present in the report + some models based on that. Proposed features:
* for top 5 popular medications - transpose to columns % of all transactions in the store per store
* top 5 manufacturers sold there - transposed
* what percent of patients have chronic disease - transposed


## Questions
1. How to treat patients that were taking diabetes meds before 2016 and later stopped (according to the dataset). Is it possible that their diabetes transactions have been held out? This is important, since we will use them as ground truth to train our classifier.
2. The whole idea of submitting probabilities instead of classes seems weird. Are the results that those will be compared against not classes? The held out set should contain classfication based on ground truth, so it should know whether a patient used diabetes drug or not. Are the probabilities transformed then into classes? I will check what exactly is the lost function. http://tiberius.biz/ausdm09/index.html#4 Seems like they measure the area under the curve obtained by plotting the cumulative % of proper classification against the misclassification (in a sense). I'm not sure though how this would produce the results shown on the leaderboard, they look more like precision (?), though this would be a bad metric for the competition :D

